In [1]:
%%time
import warnings
warnings.filterwarnings('ignore')
from collections import  namedtuple
import pandas as pd
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 5.22 s, sys: 492 ms, total: 5.71 s
Wall time: 5.12 s


In [8]:
resize_factor = [2]
unfrozen_epochs = [4, 7]
frozen_epochs = [4, 7]
base_lower_bound = [5, 7, 10, 15]
erosion_dilation_kernel_size = [20, 50, 80, 150]
ratio_split_lower_bound = [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]

In [9]:
train = list(product(resize_factor, frozen_epochs, unfrozen_epochs))
train_names = ['resize_factor', 'frozen_epochs', 'unfrozen_epochs']

post_processing = list(product(base_lower_bound, erosion_dilation_kernel_size, ratio_split_lower_bound))
post_processing_names = ['base_lower_bound', 'erosion_dilation_kernel_size', 'ratio_split_lower_bound']

all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}

detector = FastAIScreenDetector()
actual_detector = AnnotatedSampledVideoScreenDetector()

In [10]:
def filtered_by(name, used):
    return any([lecture in str(name) for lecture in used]) and 'frame' in str(name)

def split_by(name, validation):
    return any([lecture in str(name) for lecture in validation])

In [11]:
def model_selection(lectures, train_names, post_processing_names, default_filtered_by, default_split_by):
    def make_splits(lectures):
        Split = namedtuple('Split', ['train', 'valid'])
        kf = KFold(n_splits=5, shuffle=True, random_state=123)
        splits = {}
        for j, split in enumerate(kf.split(lectures)):
            train_lectures = [lectures[index] for index in split[0]]
            valid_lectures = [lectures[index] for index in split[1]]
            splits[j] = Split(train=train_lectures, valid=valid_lectures)
        return splits

    splits = make_splits(all_lectures)
    df_all = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count', 'kfold_split'])

    for train_values in tqdm(train):
        resize_factor, frozen_epochs, unfrozen_epochs = train_values
        CONFIGURATION['resize_factor'] = str(resize_factor)
        CONFIGURATION['frozen_epochs'] = str(frozen_epochs)
        CONFIGURATION['unfrozen_epochs'] = str(unfrozen_epochs)

        for j in splits.keys():
            filtered_by = partial(default_filtered_by, used=splits[j].train + splits[j].valid)
            
            valid_frames = [all_frames_grouped_by_videos[lecture][0].pathname for lecture in splits[j].valid]
            split_by = partial(default_split_by, validation=splits[j].valid + valid_frames)

            detector = FastAIScreenDetector(filtered_by=filtered_by, valid_func=split_by)
            detector.train()

            valid_frames = [frame for frame in all_frames if split_by(frame.pathname)]
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)

            for post_processing_values in post_processing:    
                preds = detector.post_processing_batch(sem_preds, valid_frames, **dict(zip(post_processing_names, post_processing_values)))
                wrong_count, ious, _ = evaluate(actuals, preds)

                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_values + post_processing_values + (iou_score, wrong_count, j)
    return df_all

In [12]:
def convert_params(best_params):
    converted_params = []
    for i, par in enumerate(best_params):
        if par.is_integer():
            converted_params.append(int(par))
        elif isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)
    
    return best_params

### Lecture-wise 5-fold cross validation

In [13]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
df_best_models = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count'])

for i, split in tqdm(enumerate(kf.split(all_lectures))):
    print(f"###################### Split No. {i}")
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    
    # Model selection
    df_all = model_selection(other_lectures, train_names, post_processing_names, filtered_by, split_by)    
    df_all['wrong_count'] = df_all['wrong_count'].astype(int)
    best_params = df_all.groupby(train_names + post_processing_names).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
    best_params = convert_params(best_params)
    best_params = dict(zip(train_names + post_processing_names, best_params))
    
    
    test_filtered_by = partial(filtered_by, used=all_lectures)
    peek_frames = [all_frames_grouped_by_videos[lecture][0].pathname for lecture in test_lectures]
    test_split_by = partial(split_by, validation=test_lectures + peek_frames)
    
    
    best_detector = FastAIScreenDetector(filtered_by=test_filtered_by, valid_func=test_split_by)
    best_detector.train(**best_params)
    
    test_frames = [frame for frame in all_frames if test_split_by(frame.pathname)]
    actuals = [actual_detector.detect(frame) for frame in test_frames]
    preds = [best_detector.detect(frame) for frame in test_frames]
    
    wrong_count, ious, _ = evaluate(actuals, preds)
    iou_score = np.nanmean(ious)
    wrong_count = len(wrong_count)
    df_best_models.loc[len(df_best_models)] = tuple(best_params.values()) + (iou_score, wrong_count)
    df_best_models.to_csv('cross_validation_results_frame_wise.csv', index=False)
    
df_best_models.to_csv('cross_validation_results_frame_wise.csv', index=False)

0it [00:00, ?it/s]
  0%|          | 0/4 [00:00<?, ?it/s]

###################### Split No. 0


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.400188,0.081220,0.975889,0.956427,0.917603,00:24
1,0.280652,0.071957,0.976002,0.943549,0.896924,00:24
2,0.160218,0.045073,0.982059,0.959140,0.923429,00:24
3,0.101575,0.029770,0.989320,0.976345,0.954380,00:25


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.058681,0.035579,0.984644,0.966295,0.936476,00:25
1,0.045995,0.019024,0.992068,0.982091,0.965094,00:26
2,0.030898,0.013583,0.994362,0.987766,0.975956,00:26
3,0.024689,0.013602,0.993985,0.987607,0.975599,00:26


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.196127,0.060425,0.981065,0.951467,0.918076,00:24
1,0.246611,0.151286,0.935092,0.846932,0.772294,00:24
2,0.164494,0.080101,0.968068,0.932557,0.886901,00:24
3,0.112573,0.059262,0.978333,0.955985,0.921937,00:24


...

epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.022163,0.037868,0.986979,0.966217,0.944026,00:27
1,0.020326,0.043804,0.984438,0.962957,0.936346,00:27
2,0.036560,0.036761,0.986628,0.969238,0.944715,00:27
3,0.022861,0.028662,0.991142,0.979289,0.961914,00:27
4,0.018239,0.039412,0.988840,0.970480,0.949172,00:27
5,0.016118,0.026961,0.991494,0.978978,0.962219,00:27
6,0.012433,0.028024,0.991420,0.978387,0.961539,00:27



100%|██████████| 4/4 [1:51:41<00:00, 1675.38s/it]


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.163155,0.145297,0.950858,0.877025,0.805278,00:26
1,0.375387,0.121898,0.959292,0.906291,0.842383,00:26
2,0.326543,0.297868,0.891792,0.678030,0.553853,00:26
3,0.261822,0.147574,0.943799,0.847490,0.760204,00:26
4,0.200710,0.111832,0.949357,0.850051,0.768298,00:26
5,0.123269,0.080328,0.970194,0.915485,0.863557,00:26
6,0.083621,0.075584,0.971484,0.919753,0.869423,00:26


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.048827,0.037884,0.984088,0.963274,0.931416,00:27
1,0.040289,0.030669,0.986749,0.970282,0.944226,00:27
2,0.035543,0.035057,0.985603,0.966493,0.937990,00:27
3,0.027807,0.025113,0.988740,0.975991,0.953919,00:27


5it [9:47:36, 7051.35s/it]
